In [1]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestIOC

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

2224

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 0, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 1, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 3, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 4, 2636.,  2.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(2224, 1701)

In [6]:
C = renko(T, 0.5)
C[:5]

[(0, 15, 2635.5, 2636.0),
 (16, 44, 2636.5, 2636.0),
 (45, 207, 2635.5, 2636.0),
 (208, 239, 2636.5, 2636.0),
 (240, 240, 2635.5, 2636.0)]

In [7]:
len(C)

36

In [8]:
signal = np.zeros(len(S.DT))
for c in C:
    tm = T.DateTimeA[c[1]] - T.DateTimeA[c[0]]
    if tm < np.timedelta64(10000, 'μs'):
        k = np.searchsorted(S.DT, T.DateTimeA[c[1]])
        signal[k] = 1 if c[3] > c[2] else - 1
        print(c, tm, k)

(208, 239, 2636.5, 2636.0) 7000 microseconds 202
(240, 240, 2635.5, 2636.0) 0 microseconds 202
(760, 760, 2637.0, 2637.5) 0 microseconds 463
(761, 781, 2638.0, 2637.5) 2000 microseconds 463
(782, 783, 2637.0, 2637.5) 1000 microseconds 464
(784, 784, 2638.0, 2637.5) 0 microseconds 464
(785, 785, 2637.0, 2637.5) 0 microseconds 464
(786, 792, 2638.0, 2637.5) 0 microseconds 464
(1362, 1362, 2638.5, 2639.0) 0 microseconds 905
(1478, 1479, 2638.5, 2639.0) 0 microseconds 929
(1480, 1485, 2639.5, 2639.0) 4000 microseconds 930
(1639, 1673, 2638.0, 2638.5) 8999 microseconds 1036
(1674, 1674, 2639.0, 2638.5) 0 microseconds 1037
(2186, 2186, 2639.5, 2639.0) 0 microseconds 1628
(2187, 2187, 2638.5, 2639.0) 0 microseconds 1629


In [9]:
trades = npBacktestIOC(S.DT.astype(np.int64), S.A, S.B, signal, 0)
trades

rec.array([( 203, 1651968067108999, 2636.45996094, 2636.41503906,  464, 1651968076222000, 2637.41992188, 2637.45996094,  1.),
           ( 465, 1651968076254000, 2637.41992188, 2637.43994141,  906, 1651968091095000, 2639.20996094, 2639.18505859, -1.),
           ( 930, 1651968091874000, 2638.95996094, 2638.85009766,  931, 1651968091906000, 2638.69995117, 2638.83496094,  1.),
           (1037, 1651968096536999, 2638.31005859, 2638.24511719, 1038, 1651968096569999, 2638.10009766, 2638.1550293 ,  1.),
           (1629, 1651968117421000, 2638.85009766, 2638.93017578, 1630, 1651968117455000, 2639.01000977, 2639.00488281, -1.)],
          dtype=[('X0', '<i8'), ('T0', '<i8'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<i8'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [10]:
# timed chart
fig = chartFigure(
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.25),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.25),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),
    Enter=dict(x=trades.T0.astype('M8[us]'), y=trades.Price0, mode='markers', 
               marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green'))                
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …

In [11]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 2},
     'opacity': 0.15,
     'type': 'rect',
     'x0': str(T.DateTimeA[int(c[0])]),
     'x1': str(T.DateTimeA[int(c[1])]),
     'y0': c[2],
     'y1': c[3]
 } for c in C]